## Sector and company size and salary

In [2]:
import pandas as pd

### columns manual
  
            'job_id', 'title', 'work_type','description', 'location','zip_code', # JOB details (location,title)
         'min_salary','max_salary', 'normalized_salary', #JOB salary
         'views',  'original_listed_time', 'expiry',  'day_posting', #JOB popularity, hiring period

         
         'is_data_related', 'formatted_experience_level','degree_level', # DATA_RELATED experience level and degree
         'job_classification',           # CLASSIFICATION business/ engineering 
         'soft_skills',                #SOFT SKILLS 
         'programming_languages_score', 'analysis_statistics_score', # HARD SKILL SCORE
         'machine_learning_score', 'data_visualization_score', 'cloud_platform_score', ## HARD SKILL SCORE
         'database_score', 'big_data_tools_score', 'big_data_score', 'business_engineering_score', # HARD SKILL SCORE

         
         'company_id', 'company_name', 'company desc', 'company_size',   # COMPANY details (size)
         'country','headquarter state', 'headquarter city',  'headquarter addr', 'headquarter zipcode',  #COMPANY locations
         'industry', 'speciality', 'employee_count', 'follower_count',  # COMPANY classfication 

In [4]:
df= pd.read_csv('/Users/morgan/Desktop/SEIS 763 ML/ML_processed_job_postings_finale_v2.csv',header=0)
df.head()

,job_id,title,work_type,description,location,zip_code,min_salary,max_salary,normalized_salary,views,...,company_size,country,headquarter state,headquarter city,headquarter addr,headquarter zipcode,industry,speciality,employee_count,follower_count
0,95428182,Administrative Coordinator,FULL_TIME,Job Title: Administrative CoordinatorOrganizat...,"Cleveland, OH",44102.0,NaN,NaN,52000.0,1.0,...,NaN,US,Ohio,Cleveland,3635 PERKINS AVE,44114,Non-profit Organizations,NaN,6,289
1,1014822088,Marketing Specialist,FULL_TIME,About TrackerTracker is a leading global cloud...,United States,NaN,70000.0,80000.0,75000.0,7.0,...,2.0,US,California,San Diego,3830 Valley Centre Dr,92130,Software Development,"CRM (Customer Relationship Management), Softwa...",75,5046
2,3189117072,Client Service Associate / Practice Manager,FULL_TIME,Company OverviewSignature Estate and Investmen...,"Redondo Beach, CA",90277.0,70000.0,100000.0,85000.0,6.0,...,2.0,US,CA,Los Angeles,2121 Avenue of the Stars,90067,Financial Services,"Investment Management, Financial Planning, Inv...",159,4650
3,3260455681,Operations Specialist,FULL_TIME,Weyerhaeuser is searching for a Operations Spe...,"Castle Rock, WA",98611.0,72742.0,109113.0,90927.5,18.0,...,7.0,US,WA,Seattle,220 Occidental Ave S,98104,Paper and Forest Product Manufacturing,"Timberlands, Wood Products, Minerals, Energy, ...",6768,92381
4,3277232283,HR & Administrative Assistant,FULL_TIME,HR & Administrative CoordinatorWork is Remote ...,United States,NaN,55000.0,72000.0,63500.0,59.0,...,NaN,US,MA,Boston,0,02109,Business Consulting and Services,NaN,12,1058


In [5]:
#df['industry'].nunique()

In [6]:
#df['industry'].unique()

In [7]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

In [8]:
ref_df= pd.read_csv('/Users/morgan/Desktop/SEIS 763 ML/Industry Classification_Sector.csv',header=0)

In [9]:
ref_df=ref_df.drop(['Industry Code','Sector Code'],axis=1)

In [10]:
ref_df.columns

Index(['industry', 'Sector'], dtype='object')

In [11]:
# Function to preprocess and tokenize text
def preprocess_text(text):
    if pd.isnull(text):
        return []
    # Tokenize and remove stopwords
    tokens = word_tokenize(text.lower())
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [word for word in tokens if word not in stop_words and word.isalnum()]
    return filtered_tokens

# Function to search for keywords in tokenized text
def keyword_search_nltk(tokens, keywords):
    return [kw for kw in keywords if kw in tokens]

In [12]:
def categorize_industry(industry, ref_df):
    industry_tokens = preprocess_text(industry)
    
    for index, row in ref_df.iterrows():
        ref_keywords = preprocess_text(row['industry'])
        if any(keyword in industry_tokens for keyword in ref_keywords):
            return row['Sector']
    
    return "Other"


# Apply the categorization function to each industry
df['sector'] = df['industry'].apply(lambda x: categorize_industry(x, ref_df))

In [13]:
df['sector'].unique()

array(['Other', 'Information Technology', 'Energy', 'Materials',
       'Financials', 'Consumer Staples', 'Industrials', 'Health Care',
       'Consumer Discretionary', 'Utilities', 'Communication Services',
       'Real Estate'], dtype=object)

In [14]:
#other=df[df['sector']=='Other']
df=df[df['sector']!='Other']
df.shape

(3902, 41)

#####################################

## model 1

In [17]:
# building a model to see sector and location
sec_df=df

In [18]:
sec_df.shape

(3902, 41)

In [19]:
# Create a DataFrame with state names and coordinates
state_coords_df= pd.read_csv('/Users/morgan/Desktop/SEIS 763 ML/State Geo Location.csv',header=0)
state_coords_df.head()

,state,name,latitude,longitude,gdp,income,region
0,AL,Alabama,32.318231,-86.902298,304936.0,276926.0,Southeast
1,AK,Alaska,63.588753,-154.493062,68056.0,52520.0,Far West
2,AZ,Arizona,34.048928,-111.093731,522767.0,464775.0,Southwest
3,AR,Arkansas,35.201050,-91.831833,178606.0,176809.0,Southeast
4,CA,California,36.778261,-119.417932,3870379.0,3166135.0,Far West


In [20]:
from geopy.geocoders import Nominatim
from geopy.distance import geodesic

In [21]:
# Split the 'items' column into multiple columns
sec_df[['city', 'state','x']] = sec_df['location'].str.split(',', expand=True)

In [22]:
sec_df.head()

,job_id,title,work_type,description,location,zip_code,min_salary,max_salary,normalized_salary,views,...,headquarter addr,headquarter zipcode,industry,speciality,employee_count,follower_count,sector,city,state,x
1,1014822088,Marketing Specialist,FULL_TIME,About TrackerTracker is a leading global cloud...,United States,NaN,70000.0,80000.0,75000.0,7.0,...,3830 Valley Centre Dr,92130,Software Development,"CRM (Customer Relationship Management), Softwa...",75,5046,Information Technology,United States,None,None
2,3189117072,Client Service Associate / Practice Manager,FULL_TIME,Company OverviewSignature Estate and Investmen...,"Redondo Beach, CA",90277.0,70000.0,100000.0,85000.0,6.0,...,2121 Avenue of the Stars,90067,Financial Services,"Investment Management, Financial Planning, Inv...",159,4650,Energy,Redondo Beach,CA,None
3,3260455681,Operations Specialist,FULL_TIME,Weyerhaeuser is searching for a Operations Spe...,"Castle Rock, WA",98611.0,72742.0,109113.0,90927.5,18.0,...,220 Occidental Ave S,98104,Paper and Forest Product Manufacturing,"Timberlands, Wood Products, Minerals, Energy, ...",6768,92381,Materials,Castle Rock,WA,None
4,3277232283,HR & Administrative Assistant,FULL_TIME,HR & Administrative CoordinatorWork is Remote ...,United States,NaN,55000.0,72000.0,63500.0,59.0,...,0,02109,Business Consulting and Services,NaN,12,1058,Energy,United States,None,None
5,3297645846,Contract Administrator,FULL_TIME,Contract AdministratorSummary of PositionThe C...,"Broomfield, CO",80020.0,74400.0,90000.0,82200.0,6.0,...,205 N AUBURN AVE,87401,Mining,NaN,205,554,Materials,Broomfield,CO,None


In [23]:
sec_df = sec_df.drop('x', axis=1)

In [24]:
# Coordinates of the Twin Cities
twin_cities_coords = (44.9778, -93.2650)

# Function to calculate the score for a state
# closer states have higher score (relocation socre), higher the closer
def calculate_score(state_coords):
    distance = geodesic(twin_cities_coords, state_coords).km
    score = 1 / (distance + 1)*10000
    return score

# Calculate scores for each state
state_coords_df['score'] = state_coords_df.apply(lambda row: calculate_score((row['latitude'], row['longitude'])), axis=1)


In [25]:
state_coords_df.head()

,state,name,latitude,longitude,gdp,income,region,score
0,AL,Alabama,32.318231,-86.902298,304936.0,276926.0,Southeast,6.621193
1,AK,Alaska,63.588753,-154.493062,68056.0,52520.0,Far West,2.339588
2,AZ,Arizona,34.048928,-111.093731,522767.0,464775.0,Southwest,5.131584
3,AR,Arkansas,35.201050,-91.831833,178606.0,176809.0,Southeast,9.145900
4,CA,California,36.778261,-119.417932,3870379.0,3166135.0,Far West,4.214518


In [26]:
#making all U.S. jobs remote, assign score 50
#using MN gdp, income

state_coords_df.loc[-1] = ['US','Remote','','',483162,416325,'Plains',50.00] 

In [27]:
state_coords_df.shape

(52, 8)

In [28]:
sec_df['state'] = sec_df['state'].str.strip()

In [29]:
# Create a dictionary to map inconsistent state names to standard abbreviations
state_mapping = {
    'California': 'CA',
    'United States': 'US',
    'South Carolina Metropolitan Area': 'SC',
    'Alabama Area': 'AL',
    'Ohio Metropolitan Area': 'OH',
    'New York Metropolitan Area': 'NY',
    'Texas Metropolitan Area': 'TX',
    'MI Area': 'MI',
    'Oregon Metropolitan Area': 'OR',
    'Nebraska Metropolitan Area': 'NE',
    'Illinois Metropolitan Area': 'IL',
    'South Carolina Area': 'SC',
    'North Carolina': 'NC',
    'Hawaii': 'HI',
    'Georgia Area': 'GA',
    'Nebraska':'NE',
    'Texas': 'TX',
    'Oregon':'OR',
    'New York': 'NY',
    'Colorado' : 'CO',
    'Kansas Metropolitan Area': 'KS',
    'LA':'CA'
}

# Map inconsistent state names to standard abbreviations in sec_df
sec_df['state'] = sec_df['state'].replace(state_mapping)

sec_df['state']=sec_df['state'].fillna('US')

In [30]:
# Merge the two DataFrames based on the 'state' column
sec_df = sec_df.merge(state_coords_df, on='state', how='left')

In [31]:
sec_df = sec_df.rename(columns={'score': 'moving_score'})

In [32]:
sec_df.describe()

,job_id,zip_code,min_salary,max_salary,normalized_salary,views,day_posting,programming_languages_score,analysis_statistics_score,machine_learning_score,...,big_data_tools_score,big_data_score,business_engineering_score,company_id,company_size,employee_count,follower_count,gdp,income,moving_score
count,3.902000e+03,3025.000000,3.674000e+03,3.674000e+03,3.902000e+03,3857.000000,3902.000000,3902.000000,3902.000000,3902.000000,...,3902.0,3902.000000,3902.0,3.902000e+03,3803.000000,3902.000000,3.902000e+03,3.902000e+03,3.902000e+03,3902.000000
mean,3.897871e+09,54587.828099,1.208576e+05,1.754383e+05,1.464125e+05,23.411978,38.613532,1.407996,1.158124,1.074065,...,0.0,0.070477,0.0,9.745660e+06,5.251906,42356.236545,1.812468e+06,1.485876e+06,1.217429e+06,16.057535
std,5.259451e+07,34116.575361,1.401390e+06,1.978785e+06,1.640213e+06,76.186034,33.183036,0.901801,1.076114,0.465579,...,0.0,0.393363,0.0,2.314799e+07,1.990553,111919.896142,5.685063e+06,1.306463e+06,1.050103e+06,17.803008
min,1.014822e+09,1002.000000,1.000000e+00,2.000000e+01,2.000050e+04,1.000000,13.000000,1.000000,0.000000,0.000000,...,0.0,0.000000,0.0,1.009000e+03,1.000000,0.000000,1.000000e+00,4.353400e+04,4.333600e+04,1.583467
25%,3.895528e+09,20701.000000,6.800000e+04,9.468680e+04,8.005006e+04,4.000000,30.000000,1.000000,1.000000,1.000000,...,0.0,0.000000,0.0,4.501000e+03,4.000000,413.250000,1.753675e+04,4.831620e+05,4.163250e+05,4.703287
50%,3.902358e+09,60045.000000,9.000000e+04,1.300000e+05,1.108235e+05,5.000000,30.000000,1.000000,1.000000,1.000000,...,0.0,0.000000,0.0,9.230900e+04,6.000000,4220.500000,1.258030e+05,8.066650e+05,6.604470e+05,6.523601
75%,3.904725e+09,90802.000000,1.200000e+05,1.768000e+05,1.460000e+05,16.000000,30.000000,1.000000,1.000000,1.000000,...,0.0,0.000000,0.0,3.053065e+06,7.000000,32781.000000,5.693270e+05,2.172010e+06,1.611169e+06,17.304814
max,3.906265e+09,99701.000000,8.500000e+07,1.200000e+08,1.025000e+08,1868.000000,191.000000,7.000000,9.000000,8.000000,...,0.0,5.000000,0.0,1.023689e+08,7.000000,748312.000000,3.270284e+07,3.870379e+06,3.166135e+06,50.000000


In [33]:
sec_df['moving_score']=sec_df['moving_score'].fillna(50.0)

In [34]:
missing_value=sec_df.isnull().sum()

In [35]:
missing_value

job_id                           0
title                            0
work_type                        0
description                      0
location                         0
zip_code                       877
min_salary                     228
max_salary                     228
normalized_salary                0
views                           45
original_listed_time             0
expiry                           0
day_posting                      0
is_data_related                  0
formatted_experience_level     973
degree_level                     0
job_classification               0
soft_skills                    338
programming_languages_score      0
analysis_statistics_score        0
machine_learning_score           0
data_visualization_score         0
cloud_platform_score             0
database_score                   0
big_data_tools_score             0
big_data_score                   0
business_engineering_score       0
company_id                       0
company_name        

In [36]:
#save to csv
sec_df.to_csv('ML_job_postings_finale_ready_v2.csv', index=False)